# 3D Human Body and Deep Learning: Demonstration.

This notebook will be dedicated to the exploration of the LIMP method for human shapes. 

### Loading pretrained network

In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

from utils_distance import *
from models.model import PointNetVAE
import trimesh
import vtk
from vtkviz.vtkVisualization import *
# More details here : https://github.com/daidedou/vtkviz/blob/main/vtkVisualization.py
from ipyvtklink.viewer import ViewInteractiveWidget
device = 'cpu'

faust_path = 'C:\\Users\\Avatar2\\Documents\\Emery\\bdd\\MPI-FAUST\\training'
faust_2500_path = "C:\\Users\\Avatar2\\Documents\\Emery\\cours\\aida\\LIMP\\faust\\FAUST"
destop_path = "C:\\Users\\Avatar2\\Desktop"
#model options
opt = lambda x:x
opt.NUM_POINTS = 2100
opt.BATCH_SIZE = 16
opt.DESC_SIZE = 512 #pointwise descriptro size after convlolutional layers
opt.LATENT_SPACE = 256 #dimension of the full (pose+style) latent space
opt.POSE_SIZE = 64 #number of dimension dedicated to pose encoding 

opt.LOCAL_TH = 0.1
opt.LEARNING_RATE = 0.1e-4

vae = PointNetVAE(opt).to(device)


#load pretrained model
net_type = '4'

loss_step = '' 
loss_step = '_ae_euc' 
loss_step = '_ae_euc_gd1' 
loss_step = '_ae_euc_gd2' 

vae.load_state_dict(torch.load('pretrained/FAUST_vae_euc_gd.dict'), strict=False)
vae.eval()

PointNetVAE(
  (enc): Encoder(
    (ptnet): SimplePointNet(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Sequential(
          (0): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (2): Sequential(
          (0): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (3): Sequential(
          (0): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (transformers): ModuleList(
      

 ### Human data 
 Let's load some human shapes!
 
 First a registered mesh.

In [25]:
def load_mesh_2500(path):
    VERT = np.loadtxt(path+'/mesh.vert')
    TRIV = np.loadtxt(path+'/mesh.triv',dtype='int32')-1
    return VERT, TRIV

In [2]:
def show_mesh_vtk(mesh):
    surf_actor = VTKSurface(mesh.vertices, mesh.faces)
    renderer = VTKVisualization()
    renderer.add_entity(surf_actor)
    renderer.init()
    ViewInteractiveWidget(renderer.window)

In [3]:
mesh_reg = trimesh.load(os.path.join(faust_path, "registrations", "tr_reg_007.ply"), process=False)
surf_actor = VTKSurface(mesh_reg.vertices, mesh_reg.faces)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

In [27]:
verts_2500, faces_2500 = load_mesh_2500(os.path.join(faust_2500_path, "tr_reg_007_2000"))
surf_actor = VTKSurface(verts_2500, faces_2500)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

Now let's load the corresponding scan mesh

In [28]:
mesh_scan = trimesh.load(os.path.join(faust_path, "scans", "tr_scan_007.ply"), process=False)
surf_actor = VTKSurface(mesh_scan.vertices, mesh_scan.faces)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

### Encoder -> Decoder 

Let's first observe the effect of the encoding of one mesh. 
The training configuration of this specific method were with meshes of size 2100 (2100 vertices), so the resolution of the reconstructed mesh will be lower

In [29]:
def encode(vae, vertices):
    vertices[:, 1] -= 0.85 + np.min(vertices[:, 1])
    verts_torch = torch.from_numpy(vertices).float().unsqueeze(0).to(device)
    lsp = vae.enc(verts_torch)
    z_latent = lsp[:, :lsp.shape[1]//2]
    return z_latent

In [30]:
z_reg = encode(vae, mesh_reg.vertices)
z_scan = encode(vae, mesh_scan.vertices)

In [31]:
print(z_reg.shape)
print(z_scan.shape)

torch.Size([1, 256])
torch.Size([1, 256])


In [ ]:
def decode(vae, latent_code):
    verts_rec = vae.dec(latent_code)
    verts_rec[:, :, 0] *= -1
    return verts_rec.squeeze().detach().cpu().numpy()

In [32]:
verts_rec_reg = decode(vae, z_reg)
surf_actor = VTKSurface(verts_rec_reg, faces_2500)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

In [33]:
verts_rec_scan = decode(vae, z_scan)
surf_actor = VTKSurface(verts_rec_scan, faces_2500)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

In [34]:
mesh_scan_2 = trimesh.load(os.path.join(faust_path, "scans", "tr_scan_012.ply"), process=False)
surf_actor = VTKSurface(mesh_scan_2.vertices, mesh_scan_2.faces)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

In [35]:
z_scan_1 = encode(vae, mesh_scan.vertices)
z_scan_2 = encode(vae, mesh_scan_2.vertices)

In [36]:
z_new = (z_scan_1 + z_scan_2)/2.

In [37]:
verts_rec_new = decode(vae, z_new)
surf_actor = VTKSurface(verts_rec_new, faces_2500)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

In [38]:
z_pose_1, z_shape_1 = z_scan_1[:, :opt.POSE_SIZE], z_scan_1[:, opt.POSE_SIZE:]
z_pose_2, z_shape_2 = z_scan_2[:, :opt.POSE_SIZE], z_scan_2[:, opt.POSE_SIZE:]

In [39]:
#z_new = torch.cat(((z_pose_1+z_pose_2)/2, z_shape_1), axis=1)
z_new = torch.cat((z_pose_2, z_shape_1), axis=1)
verts_rec_new = decode(vae, z_new)
surf_actor = VTKSurface(verts_rec_new, faces_2500)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

### Dealing with noise

What is the behavior of our network with noisy shapes?

In [40]:
mesh_noise = trimesh.load(os.path.join(destop_path, "cvssp_1.ply"), process=False)
surf_actor = VTKSurface(mesh_noise.vertices, mesh_noise.faces)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)

In [41]:
z_noise = encode(vae, mesh_noise.vertices)
verts_rec_noise = decode(vae, z_noise)
surf_actor = VTKSurface(verts_rec_noise, faces_2500)
renderer = VTKVisualization()
renderer.add_entity(surf_actor)
renderer.init()
ViewInteractiveWidget(renderer.window)

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=1200)